In [29]:
import pandas as pd
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
df_SteamGames=pd.read_csv('steam_games_cleaned.csv')
df_UserReviews=pd.read_csv('user_reviews_cleaned.csv')

In [31]:
# Se crea un nuevo DataFrame llamado df_games que contiene solo las columnas relevantes
game_columns = ['item_id', 'name']
df_games = df_SteamGames[game_columns]

# Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
df_joined = pd.merge(df_games, df_UserReviews[['item_id', 'review']], on='item_id', how='inner')

In [32]:
#Se crea una muestra para el modelo
muestra = df_joined.head(20000)

In [33]:
# Se crea el modelo de machine learning con Scikit-Learn
tfidf = TfidfVectorizer(stop_words='english')
muestra=muestra.fillna("")

tdfid_matrix = tfidf.fit_transform(muestra['review'])
cosine_similarity = linear_kernel( tdfid_matrix, tdfid_matrix)

### Función recomendacion_juego()
Ingresando el id de producto (juego) recibiremos una lista con 5 juegos recomendados similares

In [38]:
# Se crea la funcion de recomendación de 5 juegos recomendados para el juego ingresado.
def recomendacion_juego(id: int):
    if id not in muestra['item_id'].values:
        return {'mensaje': 'No existe el id del juego'}
    
    titulo = muestra.loc[muestra['item_id'] == id, 'name'].iloc[0]
    idx = muestra[muestra['name'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = muestra['name'].iloc[sim_ind].values.tolist()
    return {f'juegos recomendados para {id}': list(sim_juegos)}

In [39]:
# Ejemplo de uso del modelo 1 (2300 - DOOM II)
print(recomendacion_juego(2300))

{'juegos recomendados para 2300': ['Max Payne 3', 'RollerCoaster Tycoon® 3: Platinum', 'Insurgency', 'ARK: Survival Evolved', 'Shower With Your Dad Simulator 2015: Do You Still Shower With Your Dad']}


In [40]:
# Ejemplo de uso del modelo 2 (21000 - LEGO Batman)
print(recomendacion_juego(21000))

{'juegos recomendados para 21000': ['Toribash', 'Doom 3: BFG Edition', 'AdVenture Capitalist', 'Portal 2', 'Portal 2']}


In [37]:
muestra

,item_id,name,review
0,282010,Carmageddon Max Pack,oh where do i start dos
1,70,Half-Life,if you own any half life but not this one that...
2,70,Half-Life,this is one of the greatest games i have ever ...
3,70,Half-Life,possibly the best fps game ever
4,70,Half-Life,do i honestly need to recommend this
...,...,...,...
19995,316010,Magic Duels,magic is a great game to play with friends and...
19996,316010,Magic Duels,i was enjoying this game playing the single pl...
19997,316010,Magic Duels,i like this its got basic rules that beginners...
19998,316010,Magic Duels,yeah so far so good interface very smooth
